In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,Dense,Dropout
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

from tensorflow.keras.optimizers import Adam

from gensim.models import Word2Vec

In [3]:
cleaned_lyrics = pd.read_csv('Data/tcc_ceds_music.csv')
list(cleaned_lyrics.lyrics)[30]

'choice life freedom voice government care health cigarettes liquor shelf propaganda feed hungry beast government best govern choice people voice anslinger shady lie congress say marijuana kill hemp fibers threaten paper mill'

In [4]:
cleaned_lyrics.columns

Index(['Unnamed: 0', 'artist_name', 'track_name', 'release_date', 'genre',
       'lyrics', 'len', 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy',
       'topic', 'age'],
      dtype='object')

In [4]:
from google.cloud import storage
import os
import io
client = storage.Client()
bucket = client.get_bucket('nlp_final_data')

blob = bucket.blob('data_no_stop.csv')
content = blob.download_as_string()

df = pd.read_csv(io.BytesIO(content))

In [5]:
df.columns

Index(['Unnamed: 0', 'title', 'tag', 'artist', 'tokens_no_stop'], dtype='object')

In [6]:
# # separate the dataset into the target variable and features
X = cleaned_lyrics['lyrics']
y = cleaned_lyrics['genre']

In [16]:
# convert the lyrics to numerical vectors using bag-of-words representation
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X_vec = vectorizer.fit_transform(X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X_tok = tokenizer.texts_to_sequences(X)

max_log_length = 500
X_processed = pad_sequences(X_tok, maxlen=max_log_length)
# # convert the sparse matrix to a numpy array
# X_balanced_vec = X_balanced_vec.toarray()

In [17]:
# split the balanced data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2,stratify=y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2,stratify=y, random_state=42)

In [18]:
# Encode the class labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the integer encoded labels to one-hot encoded vectors
num_classes = len(label_encoder.classes_)
y_train_onehot = np_utils.to_categorical(y_train_encoded, num_classes)
y_test_onehot = np_utils.to_categorical(y_test_encoded, num_classes)

In [19]:
# build the neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='softmax')
])

# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model3 = Sequential([
    Embedding(input_dim=num_words,output_dim=32,input_length=max_log_length),
    LSTM(units=64,recurrent_dropout=0.5),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='softmax')
])

# compile the model
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
model3.fit(X_train,y_train_encoded,epochs=3,validation_split=0.25,batch_size=128) 

Epoch 1/3


ValueError: in user code:

    File "/Users/scottsmacbook/opt/anaconda3/envs/Capstone/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/scottsmacbook/opt/anaconda3/envs/Capstone/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/scottsmacbook/opt/anaconda3/envs/Capstone/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/scottsmacbook/opt/anaconda3/envs/Capstone/lib/python3.9/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/scottsmacbook/opt/anaconda3/envs/Capstone/lib/python3.9/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    Fi

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# Build and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.17      0.14      0.16       921
           1       0.24      0.09      0.13      1089
           2       0.08      0.08      0.08       181
           3       0.14      0.20      0.16       769
           4       0.25      0.39      0.31      1408
           5       0.16      0.14      0.15       500
           6       0.15      0.12      0.13       807

    accuracy                           0.20      5675
   macro avg       0.17      0.16      0.16      5675
weighted avg       0.19      0.20      0.18      5675



/Users/scottsmacbook/opt/anaconda3/envs/Capstone/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [59]:
# train the model

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

ValueError: Data cardinality is ambiguous:
  x sizes: 28372
  y sizes: 22697
Make sure all arrays contain the same number of samples.